In [84]:
import torch
import torch.nn as nn
import numpy as np
from util.utils import load_data_set, gen_batch_sequence, run_lstm
from transformers import BertTokenizer, BertModel
from model.word_embedding import WordEmbedding

In [85]:
train_query, train_table = load_data_set('train')

Loading train dataset
Loaded 56355 queries and 18585 tables


In [86]:
selected_query_id = [34,56,12,43] #random
ret_tup = gen_batch_sequence(train_query,train_table,selected_query_id, 0, len(selected_query_id))

In [87]:
batch_query = ret_tup[0]
batch_table = ret_tup[1]
gt_quety = ret_tup[4]
col_num = ret_tup[2]
print(batch_table[0])

[['player'], ['no', '.'], ['nationality'], ['position'], ['years', 'in', 'toronto'], ['school/club', 'team']]


In [97]:
print(batch_table)

[[['player'], ['no', '.'], ['nationality'], ['position'], ['years', 'in', 'toronto'], ['school/club', 'team']]]


In [98]:
print(gt_quety)

[['SELECT', 'position', 'FROM', 'table_', 'WHERE', 'school/club', 'team', 'EQL', 'connecticut']]


In [88]:
word_emb = WordEmbedding('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
print(word_emb.bert_model.config.hidden_size)

768


In [89]:
q_seq, col_seq, col_num, ans_seq, query_seq, ground_truth_cond_seq, raw_data = gen_batch_sequence(train_query, train_table,selected_query_id,0,len(selected_query_id))
print(raw_data)

[('During which years was Marcus Banks in Toronto?', ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'SELECT years in toronto FROM table_ WHERE player EQL marcus banks'), ('What is the canton of grande dixence?', ['Name', 'Canton', 'Height (meters)', 'Crest length (meters)', 'Type', 'Year of construction', 'Name of the Lake'], 'SELECT canton FROM table_ WHERE name EQL grande dixence'), ('What school did player number 6 come from?', ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], 'SELECT school/club team FROM table_ WHERE no . EQL 6'), ('What time was the highest for 2nd finishers?', ['Year', 'Tournaments played', 'Cuts made*', 'Wins', '2nd', 'Top 10s', 'Best finish', 'Earnings ($)', 'Money list rank', 'Scoring average', 'Scoring rank'], 'SELECT max ( 2nd ) FROM table_ WHERE')]


In [37]:
input_lengths

[9, 18, 9, 22]

In [38]:
col_length

array([ 6,  7,  6, 11], dtype=int64)

## Function for running LSTM 

In [9]:
bert_op, input_lens = word_emb.gen_x_batch(batch_query, batch_table)
x_emb = bert_op.last_hidden_state
x_emb.shape

torch.Size([4, 47, 768])

In [11]:
bert_op.pooler_output.size()

torch.Size([4, 768])

In [12]:
nn.LSTM?

In [13]:
lstm = nn.LSTM(768,hidden_size = 100//2, num_layers = 4,bidirectional=True)

lstm.to("cuda")

LSTM(768, 50, num_layers=4, bidirectional=True)

In [14]:
h_n, (c_fwd, c_rev) = lstm(x_emb)

In [15]:
print(f"h_n: {h_n.shape}\nc_wd: {c_fwd.shape}\nc_rev: {c_rev.shape}")

h_n: torch.Size([4, 47, 100])
c_wd: torch.Size([8, 47, 50])
c_rev: torch.Size([8, 47, 50])


In [12]:
#  in the code for the reference paper they sorted the inputs (acc. to the size)
# TODO: Clean this function: 
def run_lstm(lstm, inp, inp_length, prev_hidden=None):
    '''
    Input: This function takes in 3 arguments 
        lstm : the name of the lstm variable that needs to be run
        inp  : the input in the for [Batch size , num_tok, last_layer]
        inp_length: an array that contains the length of each element in the batch size = batch size
        pre_hidden: hidden layer values of the previous lstm layer
    
    Ouptut: 
        Same as nn.LSTM
    '''
    ret_h, ret_c = lstm(inp,prev_hidden)
    return ret_h, ret_c

## Attention layer

In [49]:
x = torch.tensor([[1,2,3,4,5,6]])
x = x.squeeze()
print(x.shape)


torch.Size([6])


In [16]:
x= torch.tensor([[[1,2,3,4],
                  [5,6,7,8]],
                [[9,10,11,12],
                 [13,14,15,16]],
                [[9,10,11,12],
                 [13,14,15,16]]
                ]).to('cuda')
x.shape

torch.Size([3, 2, 4])

In [17]:
num_hidden = 100

### Scalar attention score

In [13]:
scalar_attention = nn.Linear(num_hidden, 1).to("cuda")

In [54]:
att_val = scalar_attention(h_n)
att_val = att_val.squeeze()
print(att_val.shape)

torch.Size([4, 47])


In [55]:
max_x_len = max(input_lens)
for idx, num in enumerate(input_lens): # reduce the importance of 0 values
    if num < max_x_len:
        att_val[idx,num:] = -100

In [82]:
softmax = nn.Softmax(dim = 1) # Probability distribution for the attention values
att = softmax(att_val)
print(att.shape)

for x in att:
    assert int(x.sum().ceil().tolist()) == 1 

torch.Size([4, 47])


In [140]:
print(h_n.shape)
att_matrix = att.unsqueeze(2).expand_as(h_n)
print(att_matrix.shape)
K_agg = (h_n*att_matrix).sum(1)
print(K_agg.shape)

torch.Size([4, 47, 100])
torch.Size([4, 47, 100])
torch.Size([4, 100])


In [150]:
print(att_matrix[0].shape)

torch.Size([47, 100])


In [83]:
agg_out = nn.Sequential(
            nn.Linear(num_hidden,num_hidden),
            nn.Tanh(),
            nn.Linear(num_hidden, 6)
).to('cuda')

In [126]:
agg_score = agg_out(K_agg)
print(agg_score)

tensor([[-0.0695,  0.1001,  0.1088,  0.0280,  0.0561, -0.0524],
        [-0.0667,  0.0973,  0.1071,  0.0263,  0.0583, -0.0472],
        [-0.0653,  0.0935,  0.1049,  0.0239,  0.0583, -0.0464],
        [-0.0636,  0.0896,  0.1036,  0.0177,  0.0548, -0.0475]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


## Column Encoding

In [90]:
col_seq

[[['player'],
  ['no', '.'],
  ['nationality'],
  ['position'],
  ['years', 'in', 'toronto'],
  ['school/club', 'team']],
 [['name'],
  ['canton'],
  ['height', '(', 'meters', ')'],
  ['crest', 'length', '(', 'meters', ')'],
  ['type'],
  ['year', 'of', 'construction'],
  ['name', 'of', 'the', 'lake']],
 [['player'],
  ['no', '.'],
  ['nationality'],
  ['position'],
  ['years', 'in', 'toronto'],
  ['school/club', 'team']],
 [['year'],
  ['tournaments', 'played'],
  ['cuts', 'made', '*'],
  ['wins'],
  ['2nd'],
  ['top', '10s'],
  ['best', 'finish'],
  ['earnings', '(', '$', ')'],
  ['money', 'list', 'rank'],
  ['scoring', 'average'],
  ['scoring', 'rank']]]

In [32]:
cols = col_seq
ret = []
col_len = np.zeros(len(cols), dtype=np.int64)
names = []
for b, one_cols in enumerate(cols):
    names = names + one_cols
    col_len[b] = len(one_cols)

In [35]:
col_len

array([ 6,  7,  6, 11], dtype=int64)

In [34]:
len(names)

30

In [92]:
bert_encoder = 'bert-base-uncased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_encoder)
bert_model = BertModel.from_pretrained(bert_encoder)
bert_args = {'padding': 'longest','return_tensors': 'pt'}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
bert_tokenizer.batch_encode_plus?

In [301]:
def gen_col_batch(cols):
    bert_args = {'add_special_tokens': False,
                  'return_token_type_ids': False,
                  'padding': 'longest',
                  'return_attention_mask': False,
                  'return_tensors': 'pt'}
    ret = []
    col_len = np.zeros(len(cols), dtype=np.int64)
    #names = [z for x in cols for y in x for z in y]
    for x in cols:
        for y in x:
            print(y)
    t = 0
    print(len(cols))
    for x in cols:
        t+=len(x)
    print(t)
    
    for b, one_cols in enumerate(cols):
        #print(one_cols[0])
        #names.append([y for x in one_cols for y in x])
        col_len[b] = len(one_cols)
    #print(len(names))  
    for x in cols:
        for y in x:
            inp_encode = bert_tokenizer.batch_encode_plus(batch_text_or_text_pairs=,padding='longest',return_tensors= 'pt')
    #print(inp_encode['input_ids'])
    bert_output = bert_model(**inp_encode)
    embs = bert_output.last_hidden_state
    #print(names)
    
#     embs = []
#     name_len = []
#     tokenized_names = []
#     tokenized_name_len = []
#     tokenized_col_len = []
    #for x in names:
        
    
#     for x in names:
#         inp_encode = bert_tokenizer.batch_encode_plus(batch_text_or_text_pairs=x,**bert_args)
#         tokenized_names.append(inp_encode)
#         tokenized_name_len.append(inp_encode['input_ids'].shape[1])
#         tokenized_col_len.append(inp_encode['input_ids'].shape[0])
#         #print(inp_encode['input_ids'])
#         #print(tokenized_col_len)
# #         for key in inp_encode.keys():
# #             inp_encode[key] = inp_encode[key].to('cuda')
#     max_len = max(tokenized_name_len)
    
    
#     for i,x in enumerate(tokenized_names):
#         #print([102] * (max_len - tokenized_name_len[i]))
#         for j in tokenized_col_len:
#             for k in range(j):
#                 input_encode = x['input_ids'][k]
#                 concat_list = [102] * (max_len - tokenized_name_len[i])
#                 concat_tensor = torch.tensor(concat_list,dtype = torch.long)
#                 #print(concat_tensor)
#                 input_encode = torch.unsqueeze(torch.cat((input_encode,concat_tensor),dim=0),0)
#                 #print(input_encode)
#                 bert_op = bert_model(input_encode)
#                 name_inp_var = bert_op.last_hidden_state
#                 name_len.append(bert_op.last_hidden_state.shape[0])
#                 embs.append(name_inp_var)
#     max_len = max(name_len)
    
    #embs = torch.Tensor(embs).to('cuda') 

#     return embs, name_len, col_len
    return embs,[0],col_len

In [302]:
col_inp_var, col_name_len, col_len = gen_col_batch(col_seq)
# print(col_inp_var[4].shape)
# for i in range(len(col_inp_var)):
#     #if not col_inp_var[i].shape[1] == 3:
#         print(i,col_inp_var[i].shape)
# print(col_name_len)
print(col_inp_var.shape)

['player']
['no', '.']
['nationality']
['position']
['years', 'in', 'toronto']
['school/club', 'team']
['name']
['canton']
['height', '(', 'meters', ')']
['crest', 'length', '(', 'meters', ')']
['type']
['year', 'of', 'construction']
['name', 'of', 'the', 'lake']
['player']
['no', '.']
['nationality']
['position']
['years', 'in', 'toronto']
['school/club', 'team']
['year']
['tournaments', 'played']
['cuts', 'made', '*']
['wins']
['2nd']
['top', '10s']
['best', 'finish']
['earnings', '(', '$', ')']
['money', 'list', 'rank']
['scoring', 'average']
['scoring', 'rank']
4
30
torch.Size([2, 3, 768])


In [181]:
col_inp_var[0].shape

torch.Size([1, 3, 768])

In [94]:
col_inp_var[0].shape

torch.Size([4, 25, 768])

In [101]:
col_name_len

[9, 18, 9, 22]

In [102]:
col_len

array([ 6,  7,  6, 11], dtype=int64)

In [94]:
enc_lstm=nn.LSTM(input_size=768, hidden_size=100 // 2, num_layers=4, batch_first=True, 
                 dropout=0.3, bidirectional=True)

In [97]:
print(type(col_inp_var))

<class 'list'>


In [96]:
name_hidden, _ = run_lstm(enc_lstm, col_inp_var, col_name_len)

AttributeError: 'list' object has no attribute 'dim'

In [28]:
def gen_col_batch(self, cols):
        col_len = np.zeros(len(cols), dtype=np.int64)
        names = []
        for b, one_cols in enumerate(cols):
            names = names + one_cols
            col_len[b] = len(one_cols)
            
        name_inp_var = []
        name_len = []
        for x in names:
            inp_encode = self.bert_tokenizer.batch_encode_plus(batch_text_or_text_pairs=x,**self.bert_args)
            bert_op = self.bert_model(**inp_encode)
            embs = bert_op.last_hidden_state
            name_inp_var.append(embs)
            name_len.append(bert_op.last_hidden_state.shape[0])
        return name_inp_var, name_len, col_len

## Selection Prediction Class

In [ ]:
def col_name_encode(name_inp_var, col_len, enc_lstm,name_len=768):
    # Encode the columns.
    # The embedding of a column name is the last state of its LSTM output.
    name, _ = run_lstm(enc_lstm, name_inp_var,name_len)
    name_out = name_hidden[tuple(range(len(name_len))), name_len - 1]
    ret = torch.FloatTensor(len(col_len), max(col_len), name_out.size()[1]).zero_()
    if name_out.is_cuda:
        ret = ret.cuda()

    st = 0
    for idx, cur_len in enumerate(col_len):
        ret[idx, :cur_len] = name_out.data[st:st + cur_len]
        st += cur_len
    ret_var = Variable(ret)

    return ret_var, col_len

In [127]:
def col_name_encode(name_inp_var, name_len, col_len, enc_lstm):
    # Encode the columns.
    # The embedding of a column name is the last state of its LSTM output.
    ret_var, _ = run_lstm(enc_lstm, name_inp_var, name_len)
    return ret_var, col_len

In [128]:
class SelectionClausePredictor(nn.Module):
    def __init__(self,input_layer,hidden_size,num_layers,max_tok_num,gpu):
        super(SelectionClausePredictor, self).__init__()
        self.max_tok_num = max_tok_num
        self.sel_lstm = nn.LSTM(input_size=input_layer, hidden_size=hidden_size // 2,
                                num_layers=num_layers, batch_first=True,
                                dropout=0.3, bidirectional=True)
        
        self.sel_att = nn.Linear(in_features=hidden_size, out_features=1)
        self.sel_col_name_enc = nn.LSTM(input_size=input_layer, hidden_size=hidden_size // 2,
                                num_layers=num_layers, batch_first=True,
                                dropout=0.3, bidirectional=True)
        self.sel_out_K = nn.Linear(hidden_size, hidden_size)
        self.sel_out_col = nn.Linear(hidden_size, hidden_size)
        self.sel_out = nn.Sequential(nn.Tanh(), nn.Linear(hidden_size, 1))
        self.softmax = nn.Softmax()
        
        if(gpu):
            self.sel_lstm = self.sel_lstm.to('cuda')
            self.sel_col_name_enc = self.sel_col_name_enc.to('cuda')
            self.sel_att  = self.sel_att.to('cuda')
            self.softmax = self.softmax.to('cuda')
            self.sel_out_K  = self.sel_out_K.to('cuda')
            self.sel_out_col  = self.sel_out_col.to('cuda')
            self.sel_out  = self.sel_out.to('cuda')
    
    def forward(self,x_input, x_len,col_inp_var, col_name_len, col_len,col_num):
        
        B = len(x_len)
        max_x_len = max(x_len)
        
        e_col, _ = col_name_encode(col_inp_var, col_name_len, col_len, self.sel_col_name_enc)
        h_enc, _ = run_lstm(self.sel_lstm, x_input, x_len) 
       
        att_val = self.sel_att(h_n)  
        att_val = att_val.squeeze()

        for idx, num in enumerate(x_len):
            if num < max_x_len:
                att_val[idx, num:] = -100
        
        att = self.softmax(att_val)
        K_sel = (h_enc * att.unsqueeze(2).expand_as(h_enc)).sum(1)
        K_sel_expand = K_sel.unsqueeze(1)

        sel_score = self.sel_out(self.sel_out_K(K_sel_expand) + self.sel_out_col(e_col)).squeeze()
        max_col_num = max(col_num)
        
        for idx, num in enumerate(col_num):
            if num < max_col_num:
                sel_score[idx][num:] = -100

        return sel_score

In [110]:
x_emb_var, x_len = word_emb.gen_x_batch(q_seq, col_seq)
col_inp_var, col_name_len, col_len = word_emb.gen_col_batch(col_seq)

In [129]:
sel_pred = SelectionClausePredictor(768, 100, 4,200,gpu=True) 
sel_score = sel_pred(x_emb_var, x_len, col_inp_var, col_name_len, col_len, col_num)

AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'dim'